<a href="https://colab.research.google.com/github/MocT117/Another-one-/blob/master/LogisiticsKPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import plotly.graph_objects as go

st.set_page_config(page_title="Semáforo Aduana", page_icon=":bar_chart:", layout="centered")

st.title("Dashboard Semáforo Aduana")
st.write("Carga tu archivo Excel para comparar EXW Real vs Fecha entrada Aduana:")

archivo = st.file_uploader("Sube tu archivo Excel", type=["xlsx"])

if archivo:
    df = pd.read_excel(archivo)

    # Convertir a fechas
    df['EXW Real'] = pd.to_datetime(df['EXW Real'], errors='coerce')
    df['Fecha entrada Aduana'] = pd.to_datetime(df['Fecha entrada Aduana'], errors='coerce')

    # Filtrar filas válidas (con ambas fechas)
    df_validas = df[df['EXW Real'].notna() & df['Fecha entrada Aduana'].notna()].copy()

    # Calcular diferencia en días hábiles
    df_validas['Diferencia días hábiles'] = df_validas.apply(
        lambda x: np.busday_count(
            x['EXW Real'].date(),
            x['Fecha entrada Aduana'].date()
            ) if pd.notna(x['EXW Real']) and pd.notna(x['Fecha entrada Aduana']) else np.nan,
        axis=1
        )

    # Semáforo nuevo: On time <= 8 días hábiles
    def status_semáforo(dias):
        if pd.isna(dias):
            return 'Sin fecha'
        elif dias <= 8:
            return 'On time'
        elif dias > 8:
            return 'Delay'
        else:
            return 'Revisar'
    df_validas['Semáforo'] = df_validas['Diferencia días hábiles'].apply(status_semáforo)

    num_filas = len(df_validas)

    # Mostrar tabla
    st.subheader("Tabla EXW Real vs Fecha entrada Aduana")
    st.dataframe(df_validas[['EXW Real', 'Fecha entrada Aduana', 'Diferencia días hábiles', 'Semáforo']])

    # Gauge chart (opcional)
    st.subheader("Diferencia EXW Real vs Fecha entrada Aduana (días hábiles)")

    if not df_validas['Diferencia días hábiles'].dropna().empty:
        promedio_dias = df_validas['Diferencia días hábiles'].mean()
    else:
        promedio_dias = 0

    minimo = 0
    maximo = max(18, int((df_validas['Diferencia días hábiles'].max() // 4 + 1) * 4))

    tick_vals = list(range(minimo, maximo+1, 4))

    fig = go.Figure(go.Indicator(
        mode = "gauge+number",
        value = promedio_dias,
        number = {'suffix': " días hábiles", 'font': {'size': 36}},
        title = {'text': "EXW Real vs Fecha entrada Aduana"},
        gauge = {
            'axis': {'range': [minimo, maximo], 'tickmode': 'array', 'tickvals': tick_vals, 'tickfont': {'size': 16}},
            'steps': [
                {'range': [minimo, 8], 'color': "#ABEBC6"},       # Verde <= 8 días hábiles
                {'range': [8, maximo], 'color': "#F5B7B1"}        # Rojo > 8 días hábiles
            ],
            'bar': {'color': "black", 'thickness': 0.25}
            }
        ))

    st.plotly_chart(fig)
    st.markdown(
        f"Promedio considerando {num_filas} diferencias",
        unsafe_allow_html=True
        )

    st.info("Dashboard generado por Kai 🤖")

else:
    st.warning("Sube un archivo para ver el dashboard.")